<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/3)_Consigna_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cantidad de contenido por planeta fuera de la tierra en la Wikipedia.**

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

Importo librerias.

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 

Creo el sparkSession.

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi drive para obtener el archivo y luego realizar su lectura.

In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Realizo la lectura del archivo geo_tags.csv.

In [5]:
geo = spark.read.format("csv").option("header", True).load('gdrive/My Drive/TP2 - Datos/geo_tags.csv')
rddGeo = geo.rdd

In [6]:
rddGeo.take(2)

[Row(gt_id='158041', gt_page_id='4328020', gt_globe='earth', gt_primary='0', gt_lat='41.50749969', gt_lon='-5.81750011', gt_dim='1000', gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id='4670424', gt_globe='earth', gt_primary='0', gt_lat='13.73799992', gt_lon='-89.29199982', gt_dim='10000', gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

Limpio el RDD con el que voy a trabajar, es decir, filtro los None. Y a su vez, realizo el filtrado de los planetas. Ya que por enunciado debo trabajar con contenidos que esten fuera de la tierra.

In [7]:
rddGeo = rddGeo.filter(lambda x: x.gt_globe != 'earth' and x.gt_globe != None and x.gt_page_id != None).map(lambda x: (x.gt_globe))

Realizo un map para quedarme con una estructura del estilo --> ((planeta),1), para luego realizar un reduceByKey y así, obtemer la cantidad de contenidos por planeta.

***Resultado***

In [8]:
rddGeo.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).collect()

[('mars', 1438),
 ('titan', 4),
 ('callisto', 3),
 ('titania', 2),
 ('hyperion', 1),
 ('tethys', 1),
 ('mimas', 1),
 ('mercury', 561),
 ('moon', 9061),
 ('ganymede', 116),
 ('venus', 24),
 ('umbriel', 1),
 ('phobos', 20),
 ('oberon', 1),
 ('io', 2),
 ('deimos', 2),
 ('enceladus', 1)]